In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [7]:
devices = pd.read_csv("loomadevices.csv")
devices.head()

,serial,model,build
0,LOOMA001,A1,B2025
1,LOOMA002,A1,B2025
2,LOOMA003,A2,B2024
3,LOOMA004,A2,B2024
4,LOOMA005,A3,B2023


In [14]:
import pandas as pd
import qrcode
import os

# Read device info from CSV
data = pd.read_csv("loomadevices.csv")  # columns: serial, model, build

# Output folder
os.makedirs("qr_codes", exist_ok=True)
for index, row in data.iterrows():
    serial = row['serial']
    model = row['model']
    build = row['build']
    url = f"http://127.0.0.1:5000serial={serial}&model={model}&build={build}"

    qr = qrcode.make(url)
    qr.save(f"qr_codes/{serial}.png")

    print(f"Generated QR for {serial}")

Generated QR for LOOMA001
Generated QR for LOOMA002
Generated QR for LOOMA003
Generated QR for LOOMA004
Generated QR for LOOMA005


In [15]:
from flask import Flask, render_template, request
import csv
from datetime import datetime
from pymongo import MongoClient
import os

app = Flask(__name__)

# MongoDB connection
client = MongoClient('mongodb://localhost:27017/')
db = client['looma']
collection = db['scans']

# Load device info from CSV
def find_device(serial):
    with open('loomadevices.csv', newline='') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            if row['serial'] == serial:
                return row
    return None

@app.route('/', methods=['GET', 'POST'])
def index():
    device = None
    success = False

    if request.method == 'POST':
        serial = request.form['serial']
        username = request.form['username']
        location = request.form['location']

        device = find_device(serial)

        if device:
            # Create MongoDB document
            document = {
                'timestamp': datetime.utcnow(),
                'serial': serial,
                'model': device['model'],
                'build': device['build'],
                'username': username,
                'location': location
            }
            collection.insert_one(document)
            success = True

    return render_template('index.html', device=device, success=success)

if __name__ == '__main__':
    if os.environ.get('WERKZEUG_RUN_MAIN') == 'true':
        app.run(debug=True)